In [56]:
import pandas as pd
from scipy.stats import shapiro,levene,ttest_ind,mannwhitneyu
from openpyxl import Workbook
from openpyxl.styles import PatternFill

In [57]:
df = pd.read_csv('eatclub_ab_daily.csv')

In [58]:
group_column = 'Gruppe'
groups_to_compare=['doi_A','doi_B']

normality_dict = {}
homogeneity_dict = {}

dataset1 = df[df[group_column]==groups_to_compare[0]]
dataset2 = df[df[group_column]==groups_to_compare[1]]

columns_to_compare = list(df.columns[4:])

In [63]:
def normality_test(df1,df2,column):
    _, p = shapiro(df1[column])
    _, p2 = shapiro(df2[column])
    if p > 0.05 and p2 > 0.05:
        return True
    else:
        return False
    
    
def homogeneity_test(df1,df2,column):
    stat, p = levene(df1[column], df2[column])
    if p > 0.05: 
        return True
    else:
        return False


In [64]:
for column in columns_to_compare:
    normality_dict[column] = normality_test(dataset1,dataset2,column)
    homogeneity_dict[column] = homogeneity_test(dataset1,dataset2,column)

In [65]:
method = None
results = pd.DataFrame(columns=['Gruppe 1', 'Gruppe 2', 'KPI', 'Durchschnitt 1', 'Durchschnitt 2' , 'p', 'Signifikant', 'Testmethode'])
alpha = 0.05    
for key,value in normality_dict.items():
            if value == True and homogeneity_dict[key] == True:
                t, p = ttest_ind(dataset1[key], dataset2[key])
                method = 'T-Test'
            else:
                t, p = mannwhitneyu(dataset1[key], dataset2[key], alternative='two-sided')  
                method = 'Mann-Whitney U'
            results_new_row = {
            'Gruppe 1': groups_to_compare[0],
            'Gruppe 2': groups_to_compare[1],
            'KPI': key,
            'Durchschnitt 1': dataset1[key].mean(),
            'Durchschnitt 2': dataset2[key].mean(),
            'p': p,
            'Signifikant': True if p < alpha else False,
            'Testmethode': method
             }
            results_new_row = pd.DataFrame([results_new_row])
            results = pd.concat([results, results_new_row], ignore_index=True)

In [62]:
def format_durchschnitt(row):
    formatting = []
    if row['Durchschnitt 1'] > row['Durchschnitt 2']:
        formatting.extend(['background-color: green' if cell == row['Durchschnitt 1'] else 'background-color: red' for cell in row])
    else:
        formatting.extend(['background-color: green' if cell == row['Durchschnitt 2'] else 'background-color: red' for cell in row])
    return formatting

excel_file_path = 'eatclub_ab_test_output.xlsx'

# Apply the custom formatting function to each row
results_style = results.style.apply(format_durchschnitt, axis=1, subset=['Durchschnitt 1', 'Durchschnitt 2'])

# Create an Excel writer
excel_writer = pd.ExcelWriter(excel_file_path, engine='openpyxl')
excel_writer.book = Workbook()

# Write the DataFrame to the Excel file with formatting
results_style.to_excel(excel_writer, sheet_name='Results', index=False)

# Access the sheet
excel_writer.sheets['Results'].title = 'AB Test Results'

# Save the Excel file
excel_writer.save()

print(f"DataFrame with cell color formatting has been saved to {excel_file_path}")

DataFrame with cell color formatting has been saved to eatclub_ab_test_output.xlsx
